# `Lib-INVENT Datasets`: Data preparation and statistics demo
This demo illustrates how to compute the distribution of the chemical properties and/or filter molecule data in SMILES format to only include drug-like molecules. The second tutorial provided in this repository demonstrates the process of slicing the filtered dataset according to reaction rules to prepare it for the training of the decorator model.

The specific parameters of the configuration correspond to those used in the preparation of the dataset used for training of the Lib-INVENT model.


### Motivation
> **There are a number of reasons to pre-process the data used for training a generative model.**
1. Removal of invalid or duplicated entries.
2. Removal of unusual compounds that are clearly not drug-like (too big, reactive groups and etc.). There is normally no point training model on such examples since that bias will reflected by the generative model. 
3. Removal of rare tokens. There are rare compounds that can be seen as outliers. They in turn might contain rare tokens. Excluding them frees a slot in the vocabulary and makes it smaller. Smaller vocabulary means faster training and less memory. As a result removing compounds that introduce rare tokens to the vocabulary speeds up the generative model.

### Introduction
This configuration can be used for preparing data to only include drug-like molecules or calculate stats for sliced datasets. This Demo mainly focuses on preparing and filtering data.
> **The rules used for filtering data:**
- 2 <= num heavy atoms <= 70   
- allowed elements: [6, 7, 8, 9, 16, 17, 35]  
- remove salts, neutralize charges, sanitize
- remove side chains with 5 or more carbon atoms
- 0<num_rings <= 10
- num_atoms >= 6
- mol_weights <= 760
- num_aromatic_rings <= 8
- heteroatom_ratio > 0.5

In [6]:
# load dependencies
import os
import re
import json
import tempfile
import pandas as pd
from pathlib import Path


In [2]:
org_data_path = "/path/to/original/data.csv"
single_col_data_path = "/path/to/your/single_col_data.csv"

df = pd.read_csv(org_data_path)
single_col_df = pd.DataFrame(df['Smiles'])
single_col_df.to_csv(single_col_data_path, index=False, header=False)

In [12]:
Path(single_col_data_path).with_suffix('').__str__()

'/home/tjosh/Lib-INVENT-dataset/datasets/libinvent/small_chembl_1k_single_col'

In [13]:

# --------- change these path variables as required
data_path = single_col_data_path # for the training of Lib-INVENT, we used ChEMBL 27 and converted to SMILES using RDKit.
output_directory = Path(single_col_data_path).with_suffix('').__str__()

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()  

# if required, generate a folder to store the results
try:
    os.mkdir(output_directory)
except FileExistsError:
    pass

## Setting up the configuration
`Lib-INVENT datasets` has an entry point that loads a specified `JSON` file on startup. `JSON` is a low-level data format that allows to specify a fairly large number of parameters in a cascading fashion very quickly. The parameters are structured into *blocks* which can in turn contain blocks or simple values, such as *True* or *False*, strings and numbers. In this tutorial, we will go through the different blocks step-by-step, explaining their purpose and potential values for given parameters. Note, that while we will write out the configuration as a `JSON` file in the end, in `python` we handle the same information as a simple `dict`.

### 1. Run Type block
The first block to add to the configuration specifies the `run_type`, which determines what operations should be performed. A complete list of the run types implemented can be found in the Lib-INVENT-Dataset>data_preparation>emuns>running_mode_enum.py script.

The aim of this tutorial is to demonstrate the process of data purging. Since it is based on the chemical properties of the compounds, the filtering is implemeted within the `stats_extraction` run type. This choice has been made for efficiency purposes as computing the properties of the large training datasets becomes very expensive and thus should optimally not need to be performed multiple times.

In [14]:
# initialize the dictionary
configuration = {
    "run_type": "stats_extraction"                                          
}

### 2. Parameters block

The purpose of the second block of the configuration is to specify the parameters according to which the operation should be performed.

As a first step, the appropriate paths to data have to be specified.
- `data_path` gives a path to the input dataset. <br> 
-- if `mode` is set to `sliced_data`, this is either a tab separated file containing the sliced dataset (with the entries on each line corresponding to scaffold, decorations separated by the separator token "|" and the original complete compound)<br>
-- if `mode` is set to `orig_data`, single column data of unsliced SMILES strings is expected.
- `output_path` specifies a directory in which all the output is saved. If the flder does not exist, it is created.
- `columns` takes a list of strings as an argument and specifies which columns are analysed. For single column data, the expected input is `original`. For sliced data,`scaffolds` and `decorations` may also be passed in the string.

In [15]:
parameters = {
    "mode": "orig_data",
    "data_path": data_path,                 # location to store input data      
    "output_path": output_directory,              # directory where the output is saved
    "columns": ["original"]
}

Next, the properties to be computed are specified. The distribution of each of the computed properties (in the form of a histogram or frequency data) is saved to the output folder. 
- `properties` takes a list of molecular properties to be computed for every compound in the analysed column(s). The possible inputs are `['mol_wts', 'num_rings','num_aromatic_rings', 'num_atoms', 'hbond_donors', 'hbond_acceptors', 'hetero_atom_ratio', 'num_tokens'].`
- `token_distribution` is a boolean parameter specifying whether a histogram of tokens present in the dataset is computed. This is computationally expensive so it isrecommended to skip if not required.
- `token_atom_ratio` is an indicator of how complicated the SMILES string is (since compliated molecules contain many brackets and other special symbols, which increases the ratio). The parameter takes boolean input specifying wther this should be computed as it can be computationally expensive.
- `count_decorations` is a boolean parameter relevant for sliced data. The number of decorations per scaffold can be computed to determine the distribution of the numbers of attachment points resulting from slicing.
- If `plotting` is set to `True`, the distribution of each property is plotted and saved.

In [16]:
parameters.update({
    "properties": ['mol_wts',               
                   'num_rings',              
                   'num_aromatic_rings',     
                   'num_atoms',
                   'hbond_donors',
                   'hbond_acceptors',
                   'hetero_atom_ratio'],
    "token_distribution": True,    
    "token_atom_ratio": True,
    "count_decorations": False,               # For sliced data only.
    "plotting": True
})

In the `orig_data` mode, the data is always standardised upon loading. This is performed using the RDKitStandardizer provided by the `reinvent-chemistry` package.

`Standardization_config` contains rules to standardise the molecules using functions in reinvent chemistry.  The default filter, as implemented in version 0.0.32 of the package, was used for the preprocessing in the Lib-INVENT publication. This standardization includes:
- 2 <= num heavy atoms <= 70
- allowed elements: [6, 7, 8, 9, 16, 17, 35]
- remove salts, neutralise charges, sanitize
- remove side chains with 5 or more carbon atoms

To save the standardised dataset, set the option `save_standardised` to `True`.

In [18]:
parameters.update({
    "standardisation_config":{},            # No argument amounts to a default filter
    "save_standardised": True,  
})

The final part of the parameters block of the configuration specifies the filtering rules, if applied.
- `filter` takes a dictionary as a parameter. An empty dictionary indicates no filtering; otherwise, the keys correspond to the properties while the values are lists specifying the filter criteria. The first entry of the list is either `max` or `min`, determining which boundary is applied (corresponding to 'greater/smaller than *or equal to* the value). The second entry is the value of the boundary. <br>
-- if two-sided bounds are to be imposed, they should be passed as two separate filters.<br>
-- any property passed as a key to the filter must have been previously calculated. If it is missing in the `properties` list, the filter reports an error.
- `save_cut_precomuted` is a boolean parameter specifying whether the filtered datset should be saved. This is typically the case for data purging.


In [19]:
parameters.update({
    "filter": {                             
        "num_rings": ["max", 10],           
        "num_rings": ["min", 1],
        "num_atoms": ["min", 6],
        "mol_wts": ["max", 760],
        "num_aromatic_rings": ["max", 8],
        "hetero_atom_ratio": ["min", 0.5],
        "token_atom_ratio": ["max", 2]
    },                                    
                                                          
    "save_cut_precomputed": True
})

### Assemble and save the configuration

With theparameters block prepared, the configuration can be assembled and saved as a JSON file.

In [20]:
configuration.update({"parameters": parameters})

In [25]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_directory, "small_chembl_1k_single_col_stats_extraction_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys= False)

## Run
Execute in jupyter notebook

In [166]:
%%capture captured_err_stream --no-stderr 

# execute
%cd </path/to/Lib-DESIGN-datasets/project/directory/>
!spark-submit --driver-memory=32g --conf spark.driver.maxResultSize=16g input.py {configuration_JSON_path}

In [167]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_directory, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

Execute in command line
```
# activate envionment
conda activate lib_invent_data

# go to the root folder of input.py 
cd </path/to/Lib-INVENT-datasets/directory>

# execute in command line
spark-submit --driver-memory=32g --conf spark.driver.maxResultSize=16g input.py </path/to/configuration.json>
```

# Reproducing the Data Purging from Lib-INVENT

The preprocessing of the ChEMBL 27 dataset for the use with the Lib-INVENT project has been performed in multiple stages. Upon each iteration, the distribution of the resulting dataset was examined and more filters imposed as necessary to remove outliers. This process amounted to gradually implementing all of the filters as given in the input above.

In the final stage, the remaining non-drug like compounds were removed manually from the command line, using the `grep` command. The following patterns were removed since they were undesirable and too rare to be learnt by the model:

`N=[N+]=[N-], [N-]=[N+]=N, [o+], [C-], [C+], %10, [c+], [cH-], [NH-], [c-], [O+], [CH-], [CH], [N], [SH2], [CH2]`.